#**Big Data Application in E-commense**
##——Customer Behavior analysis and recommendation
###Project Contributor : **Tao Liu**



In this project, we will use [Amazon Review Data](http://deepyeti.ucsd.edu/jianmo/amazon/index.html) and perform different kind of data anlysis methods to analyze the customer behaviors for buying good and then give recommendation based on anlyze. 

##**Step 0** - Package import
All the packages will be imported here.

In [1]:
import numpy
import sklearn
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

##**Step 1** - Data Implemtation and cleaning

We will start with Amazon Review Data implementation and cleaning them if possible.

